# Aula 17 - CNN + Transfer Learning

In [10]:
# Imports
import tensorflow as tf
import numpy as np

In [11]:
# Carregar o dataset
import tensorflow_datasets as tfds

(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"],
    batch_size=-1, 
    as_supervised=True
)

In [12]:
train_ds.shape

TensorShape([2569, 442, 1024, 3])

In [13]:
test_ds.shape

TensorShape([1101, 442, 1024, 3])

## Processar os dados

In [14]:
size = (150 , 150)

train_ds = tf.image.resize(train_ds, size)
test_ds = tf.image.resize(test_ds, size)

train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=5)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=5)

In [15]:
train_labels

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

## Modelo "profundo" simples

In [16]:
# Import 
from keras import Sequential, layers
from keras.callbacks import EarlyStopping
from keras.layers.experimental.preprocessing import Rescaling

# Definir a arquitetura da CNN -> camada de extração de features
hand_model = Sequential()
hand_model.add( Rescaling(1./255, input_shape=(150, 150, 3)) )

hand_model.add( layers.Conv2D(16, kernel_size=10, activation="relu") )
hand_model.add( layers.MaxPooling2D(3) ) 

hand_model.add( layers.Conv2D(32, kernel_size=8, activation="relu") )
hand_model.add( layers.MaxPooling2D(2) ) 

hand_model.add( layers.Conv2D(32, kernel_size=6, activation="relu") )
hand_model.add( layers.MaxPooling2D(2) ) 

# Definir a arquitetura da CNN -> camada de decisão (classificação)
hand_model.add( layers.Flatten() )
hand_model.add( layers.Dense(50, activation="relu") )
hand_model.add( layers.Dense(20, activation="relu") )
hand_model.add( layers.Dense(5, activation="softmax") )

# Compila o modelo
hand_model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Definir a estratégia de Early Stopping
es = EarlyStopping(monitor="val_accuracy", 
                   patience=5, 
                   restore_best_weights=True)

# Treinamento do modelo 
with tf.device("/device:GPU:0"):
  hand_model.fit(train_ds, train_labels,
                 epochs=50,
                 validation_split=0.2,
                 batch_size=32,
                 callbacks=[es])

ModuleNotFoundError: No module named 'keras.layers.experimental'

In [ ]:
# Avaliação do modelo
loss, acc = hand_model.evaluate(test_ds, test_labels)

print(f"A loss do modelo é {loss:.2f} e a ACC é: {acc:.2f}")

In [ ]:
# Michel TODO - trazer a matriz de confusão e gráficos de loss/accuracy

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_test, y_pred):
    labels = list(map(np.argmax, y_test))
    labels_pred = list(map(np.argmax, y_pred))

    cf_matrix = confusion_matrix(labels, labels_pred)
    sns.heatmap(cf_matrix, annot=True)

In [ ]:
preds = hand_model.predit(test_ds)
plot_confusion_matrix(test_labels, preds)

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

base_model = VGG16(weights="imagenet", include_top=False)

In [ ]:
base_model.summary()